# @tf.function ：图执行模式
https://tf.wiki/zh/basic/tools.html#tf-function

虽然默认的即时执行模式（Eager Execution）为我们带来了灵活及易调试的特性，但在特定的场合，例如追求高性能或部署模型时，我们依然希望使用 TensorFlow 1.X 中默认的图执行模式（Graph Execution），将模型转换为高效的 TensorFlow 图模型。此时，TensorFlow 2 为我们提供了 tf.function 模块，结合 AutoGraph 机制，使得我们仅需加入一个简单的 @tf.function 修饰符，就能轻松将模型以图执行模式运行。

## @tf.function 基础使用方法 
在 TensorFlow 2 中，推荐使用 @tf.function （而非 1.X 中的 tf.Session ）实现图执行模式，从而将模型转换为易于部署且高性能的 TensorFlow 图模型。只需要将我们希望以图执行模式运行的代码封装在一个函数内，并在函数前加上 @tf.function 即可，如下例所示。关于 TensorFlow 1.X 版本中的图执行模式可参考 附录 。

In [1]:
import tensorflow as tf
import numpy as np
import time
#from zh.model.mnist.cnn import CNN
#from zh.model.utils import MNISTLoader


class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [2]:
num_batches = 400
batch_size = 50
learning_rate = 0.001
data_loader = MNISTLoader()

@tf.function
def train_one_step(X, y):    
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        # 注意这里使用了TensorFlow内置的tf.print()。@tf.function不支持Python内置的print方法
        tf.print("loss", loss)  
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

if __name__ == '__main__':
    model = CNN()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    start_time = time.time()
    for batch_index in range(num_batches):
        X, y = data_loader.get_batch(batch_size)
        train_one_step(X, y)
    end_time = time.time()
    print(end_time - start_time)      

loss 2.3127439
loss 2.2556479
loss 2.14193726
loss 1.96743441
loss 1.79970121
loss 1.50606477
loss 1.37653339
loss 1.31806219
loss 1.13116229
loss 0.851478875
loss 0.910327
loss 0.961535096
loss 0.787063122
loss 0.723145902
loss 0.499469072
loss 0.81531173
loss 0.839180827
loss 0.741758823
loss 0.601519823
loss 0.619891644
loss 0.532553852
loss 0.635587215
loss 0.628205061
loss 0.762837231
loss 0.453942865
loss 0.75028044
loss 0.354831189
loss 0.333435565
loss 0.219041973
loss 0.335987687
loss 0.322120428
loss 0.500030816
loss 0.241591871
loss 0.262259722
loss 0.319545656
loss 0.251862019
loss 0.460616976
loss 0.256505877
loss 0.450339228
loss 0.274512291
loss 0.255688816
loss 0.246551648
loss 0.278954208
loss 0.299235731
loss 0.152727857
loss 0.189101011
loss 0.116796158
loss 0.122517258
loss 0.198961586
loss 0.3058002
loss 0.171655387
loss 0.253458589
loss 0.394212902
loss 0.347200274
loss 0.18580471
loss 0.275542945
loss 0.179178655
loss 0.0748097897
loss 0.213784426
loss 0.12851424

运行 400 个 Batch 进行测试，加入 @tf.function 的程序耗时 35.5 秒，未加入 @tf.function 的纯即时执行模式程序耗时 43.8 秒。可见 @tf.function 带来了一定的性能提升。一般而言，当模型由较多小的操作组成的时候， @tf.function 带来的提升效果较大。而当模型的操作数量较少，但单一操作均很耗时的时候，则 @tf.function 带来的性能提升不会太大。

### @tf.function 内在机制 
当被 @tf.function 修饰的函数第一次被调用的时候，进行以下操作：

在即时执行模式关闭的环境下，函数内的代码依次运行。也就是说，每个 tf. 方法都只是定义了计算节点，而并没有进行任何实质的计算。这与 TensorFlow 1.X 的图执行模式是一致的；

使用 AutoGraph 将函数中的 Python 控制流语句转换成 TensorFlow 计算图中的对应节点（比如说 while 和 for 语句转换为 tf.while ， if 语句转换为 tf.cond 等等；

基于上面的两步，建立函数内代码的计算图表示（为了保证图的计算顺序，图中还会自动加入一些 tf.control_dependencies 节点）；

运行一次这个计算图；

基于函数的名字和输入的函数参数的类型生成一个哈希值，并将建立的计算图缓存到一个哈希表中。

在被 @tf.function 修饰的函数之后再次被调用的时候，根据函数名和输入的函数参数的类型计算哈希值，检查哈希表中是否已经有了对应计算图的缓存。如果是，则直接使用已缓存的计算图，否则重新按上述步骤建立计算图。

以下是一个测试题：

In [3]:
import tensorflow as tf
import numpy as np

@tf.function
def f(x):
    print("The function is running in Python")
    tf.print(x)

a = tf.constant(1, dtype=tf.int32)
f(a)
b = tf.constant(2, dtype=tf.int32)
f(b)
b_ = np.array(2, dtype=np.int32)
f(b_)
c = tf.constant(0.1, dtype=tf.float32)
f(c)
d = tf.constant(0.2, dtype=tf.float32)
f(d)


The function is running in Python
1
2
2
The function is running in Python
0.1
0.2


当计算 f(a) 时，由于是第一次调用该函数，TensorFlow 进行了以下操作：

将函数内的代码依次运行了一遍（因此输出了文本）；

构建了计算图，然后运行了一次该计算图（因此输出了 1）。这里 tf.print(x) 可以作为计算图的节点，但 Python 内置的 print 则不能被转换成计算图的节点。因此，计算图中只包含了 tf.print(x) 这一操作；

将该计算图缓存到了一个哈希表中（如果之后再有类型为 tf.int32 ，shape 为空的张量输入，则重复使用已构建的计算图）。

计算 f(b) 时，由于 b 的类型与 a 相同，所以 TensorFlow 重复使用了之前已构建的计算图并运行（因此输出了 2）。这里由于并没有真正地逐行运行函数中的代码，所以函数第一行的文本输出代码没有运行。计算 f(b_) 时，TensorFlow 自动将 numpy 的数据结构转换成了 TensorFlow 中的张量，因此依然能够复用之前已构建的计算图。

计算 f(c) 时，虽然张量 c 的 shape 和 a 、 b 均相同，但类型为 tf.float32 ，因此 TensorFlow 重新运行了函数内代码（从而再次输出了文本）并建立了一个输入为 tf.float32 类型的计算图。

计算 f(d) 时，由于 d 和 c 的类型相同，所以 TensorFlow 复用了计算图，同理没有输出文本。

而对于 @tf.function 对 Python 内置的整数和浮点数类型的处理方式，我们通过以下示例展现：



In [4]:
f(d)
f(1)
f(2)
f(1)
f(0.1)
f(0.2)
f(0.1)

0.2
The function is running in Python
1
The function is running in Python
2
1
The function is running in Python
0.1
The function is running in Python
0.2
0.1


下一个思考题：

In [5]:
import tensorflow as tf

a = tf.Variable(0.0)

@tf.function
def g():
    a.assign(a + 1.0)
    return a

print(g())
print(g())
print(g())
print(a)

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(3.0, shape=(), dtype=float32)
<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>


正如同正文里的例子一样，你可以在被 @tf.function 修饰的函数里调用 tf.Variable 、 tf.keras.optimizers 、 tf.keras.Model 等包含有变量的数据结构。一旦被调用，这些结构将作为隐含的参数提供给函数。当这些结构内的值在函数内被修改时，在函数外也同样生效。

## AutoGraph：将 Python 控制流转换为 TensorFlow 计算图 

前面提到，@tf.function 使用名为 AutoGraph 的机制将函数中的 Python 控制流语句转换成 TensorFlow 计算图中的对应节点。以下是一个示例，使用 tf.autograph 模块的低层 API tf.autograph.to_code 将函数 square_if_positive 转换成 TensorFlow 计算图：

In [6]:
import tensorflow as tf

@tf.function
def square_if_positive(x):
    if x > 0:
        x = x * x
    else:
        x = 0
    return x

a = tf.constant(1)
b = tf.constant(-1)
print(square_if_positive(a), square_if_positive(b))
print(tf.autograph.to_code(square_if_positive.python_function))

tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32)
def tf__square_if_positive(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('square_if_positive', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def if_true():
      x_1, = x,
      x_1 = x_1 * x_1
      return x_1

    def if_false():
      x = 0
      return x
    cond = x > 0
    x = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('x',), ())
    do_return = True
    retval_ = fscope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



我们注意到，原函数中的 Python 控制流 if...else... 被转换为了 x = ag__.if_stmt(cond, if_true, if_false, get_state, set_state) 这种计算图式的写法。AutoGraph 起到了类似编译器的作用，能够帮助我们通过更加自然的 Python 控制流轻松地构建带有条件 / 循环的计算图，而无需手动使用 TensorFlow 的 API 进行构建。

## 使用传统的 tf.Session

不过，如果你依然钟情于 TensorFlow 传统的图执行模式也没有问题。TensorFlow 2 提供了 tf.compat.v1 模块以支持 TensorFlow 1.X 版本的 API。同时，只要在编写模型的时候稍加注意，Keras 的模型是可以同时兼容即时执行模式和图执行模式的。注意，在图执行模式下， model(input_tensor) 只需运行一次以完成图的建立操作。

例如，通过以下代码，同样可以在 MNIST 数据集上训练前面所建立的 MLP 或 CNN 模型：

关于图执行模式的更多内容可参见 [图执行模式下的 TensorFlow](https://tf.wiki/zh/appendix/static.html)。